In [469]:
#imports
import pandas as pd 
from slugify import slugify
import requests
from tqdm.notebook import tqdm
from time import sleep
from bs4 import BeautifulSoup as bs
import json
from datetime import datetime
import webbrowser
from tqdm.notebook import tqdm

In [470]:
#getting both csv files 
movie_details = pd.read_csv('/Users/armaan/src/bbox/mdata/thenum/movie_details.csv') 
imdb_movies = pd.read_csv('/Users/armaan/src/bbox/mdata/imdb/imdb_movies.csv') 
#converting names to lower and some cols to int
movie_details['name_lower'] = movie_details['name'].str.lower()
imdb_movies['year'] = pd.to_numeric(imdb_movies.startYear, errors='coerce')
imdb_movies['primaryTitle_lower'] = imdb_movies['primaryTitle'].str.lower()
imdb_movies['runtimeMinutes'] = pd.to_numeric(imdb_movies.runtimeMinutes, errors='coerce')

In [471]:
#normalization
def normalize(x):
    if x:
        #for star wars
        x = x.replace('-ep-', '-episode-')
        #for harry potter
        x = x.replace('part-ii', 'part-2')
        x = x.replace('part-i', 'part-1')
        #for e.t. only!
        if x.startswith('et-'):
            x = x.replace('et-', 'e-t-')
        #for rise of skywalker only!
        if x == 'star-wars-the-rise-of-skywalker':
            x = "star-wars-episode-ix-the-rise-of-skywalker"
    return x

#find duplicates
def duplicate_scraper(imdb_id):
    r = requests.get("https://www.imdb.com/title/{}/".format(imdb_id))
    soup = bs(r.text)
    if soup.find("h3",{"class": "subheading"}).string == "Box Office":
        drop = False
    else:
        drop = True 
    return drop

In [472]:
#sluggifying and normalizing all names
movie_details['name_slug'] = movie_details.name_lower.apply(slugify)
imdb_movies['name_slug'] = imdb_movies.primaryTitle_lower.apply(slugify)
movie_details['name_slug'] = movie_details.name_slug.apply(normalize)
#creating new df with updated values
new_df = pd.merge(movie_details, imdb_movies, how ='inner', right_on = ['name_slug', 'year'], left_on = ['name_slug', 'year'])

In [473]:
#find duplicates and get rid of them
duplicates = new_df[new_df.name.duplicated()]
for index, row in tqdm(duplicates.iterrows()):
    match = new_df[(new_df['name'] == row["name"]) & (new_df['year'] == row["year"])]
    if len(match) > 1:
        imdb_ids = match["tconst"]
        for imdb_id in imdb_ids:
            drop = duplicate_scraper(imdb_id)
            if drop == True:
                new_df.drop( new_df[ new_df['tconst'] == imdb_id ].index , inplace=True)

In [474]:
#drops
new_df.drop(["endYear", "isAdult", "titleType", "startYear", "primaryTitle_lower", "runtimeMinutes", "genres", "name_lower", "name_slug", "primaryTitle", "originalTitle"], axis=1, inplace = True)

In [475]:
#get poster path and add it to new col in df
poster_link_list = []
for p in tqdm(new_df['tconst']):
    tmdb = requests.get("https://api.themoviedb.org/3/movie/{}?api_key=a6837e56522d89b354d01b07e393bebc&language=en-US".format(p))
    poster_path = tmdb.json()['poster_path']
    poster_link = "https://image.tmdb.org/t/p/w300_and_h450_bestv2/{}".format(poster_path)
    poster_link_list.append(poster_link)

new_df["poster_path"] = poster_link_list

In [476]:
#downloads df to csv
new_df.to_csv('/Users/armaan/src/bbox/mdata/master_table.csv', index=False)